In [ ]:
from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer
# import other libraries
import pandas as pd
import numpy as np
from scipy.interpolate import Rbf
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import lavavu
%load_ext autoreload

### Load and process the data
We need to create a dataset that has the following columns: **type, X, Y, Z**, val, strike, dip, polarity, nx, ny, nz.
The bold columns are required and the unbold ones are optional.

Val should be the relative scalar field value of the contacts from a reference horizon. If the base of a series is 0, val should be the accumulated thickness. 

In this example we will build the scalar field for one series. The order and names of the different units in the series components were extracted map2loop topology analysis and are for now hardcoded.  

In [ ]:
df = pd.read_csv('../test_data3/output/formation_thicknesses.csv')

In [ ]:
thickness = {}
for f in df['formation'].unique():
    thickness[f] = np.mean(df[df['formation']==f]['thickness'])

display(thickness)

In [ ]:
order = ['P__TKa_xs_k','P__TKo_stq','P__TKk_sf','P__TK_s',
'A_HAu_xsl_ci',
'A_HAd_kd',
'A_HAm_cib',
'A_FOj_xs_b',
'A_FO_xo_a',
'A_FO_od',
'A_FOu_bbo',
'A_FOp_bs',
'A_FOo_bbo',
'A_FOh_xs_f',
'A_FOr_b']

In [ ]:
strat_val = {}
val = 0
for o in order:
    if o in thickness:
        strat_val[o] = val
        val+=thickness[o]
        
display(strat_val)

Now add the stratigraphic value calculated by accumulating the thickness to the contacts dataset.

In [ ]:
orientations = pd.read_csv('../test_data3/output/orientations.csv')
contacts = pd.read_csv('../test_data3/output/contacts4.csv')

In [ ]:
# orientations = orientations.rename(columns={"formation":"type"})
# contacts = contacts.rename(columns={"formation":"formatype"})
contacts['val'] = np.nan 

for o in strat_val:
    contacts.loc[contacts['formation']==o,'val'] = strat_val[o]
data = pd.concat([orientations,contacts],sort=False)
data['type'] = np.nan
for o in order:
    data.loc[data['formation']==o,'type'] = 's0'
data 

In [ ]:
from pyamg import solve
def solve_pyamg(A,B):
    return solve(A,B,verb=False,tol=1e-8)


## Set up loop structural geological model

In [ ]:
boundary_points = np.zeros((2,3))
boundary_points[0,0] = 500000.0000000026#origin[0]#-1000#-10 #np.min(coords[:,0])-10
boundary_points[0,1] = 7454093.693147668#origin[1]#-1000#-10 #np.min(coords[:,1])
boundary_points[0,2] = -20000#origin[2]#-4500#-5000#0#-40#minz
boundary_points[1,0] = 605065.8521#maximum[0]#10000#10000 #np.max(coords[:,0])
boundary_points[1,1] = 7567173.042479938#maximum[1]#10000#12000 #np.max(coords[:,1])
boundary_points[1,2] = 500#maximum[2]#500#500#-50000#-minz*0.1

In [ ]:
model = GeologicalModel(boundary_points[0,:],boundary_points[1,:])
model.set_model_data(data)
strati = model.create_and_add_conformable_foliation('s0', #identifier in data frame
                                                    interpolatortype="FDI", #which interpolator to use
                                                    nelements=400000, # how many tetras/voxels
                                                    buffer=0.1, # how much to extend nterpolation around box
                                                    solver='external',
                                                    external=solve_pyamg
                                                   )

In [ ]:
viewer = LavaVuModelViewer()
viewer.add_data(strati)
viewer.add_isosurface(strati,
#                       nslices=10,
                      slices= strat_val.values(),
#                     voxet={'bounding_box':boundary_points,'nsteps':(100,100,50)},
                      paint_with=strati,
                      cmap='tab20'

                     )
viewer.add_scalar_field(model.bounding_box,(100,100,100),
                          'scalar',
#                             norm=True,
                         paint_with=strati,
                         cmap='tab20')
viewer.set_viewer_rotation([-53.8190803527832, -17.1993350982666, -2.1576387882232666])
viewer.save("fdi_surfaces.png")
viewer.interactive()